- 전처리 어떻게 할까?

[ARTPRICE]
--> Title : 유지
--> Author
1. 순서대로 1,2,3,...,29,30 이렇게 붙이면?
--> year
1. 결측치 있는 것은 어떻게?
--> standard(그림크기)
1. 가로*세로 곱한 값을 가져와서 '넓이'로 만든다.
2. 결측치 있는 것은 drop?
--> genre
1. 회화 / 판화 / 서예 / 서예-현판 / 입체 / 공예 / 사진-미디어 / 기타 로 구분하여 각각에 0,1,2,3... 식의 값을 부여
--> technique
1. 요소들이 'AAAA, BBBB, CCCC' 형태로 되어 있는 것들이 많아서 이것들을 다 분리해야 함
2. 분리한 이후 각 요소들을 포함하는지의 여부를 one-hot encoding으로 정리
3. 결측치 있는 것은 어떻게?
--> source
1. **옥션, **옥션 온라인으로 구분됨

1-1. 각각의 source를 그냥 0,1,2...로 값을 준다.

1-2. 온라인 여부를 따로 column을 만들고, K옥션 온라인과 K옥션을 통합하여 전처리한다.

--> price
1. 원화 표시, 콤마 표시 제거

[Economic]
- 주가, 비트코인, 유가
: 거래 당일날의 가격에 해당하는 종가 가격을 찾아 join
- 부동산 가격
: 거래 당월에 해당하는 가격상승 값을 찾아 join

[Correlation]
- 각 요소들과 price와의 상관관계를 linechart, heatmap 등으로 표현하여 시각화


In [174]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [175]:
df = pd.read_excel('./crawling_files/artprice_total.xlsx', engine='openpyxl')
df.head()

,title,author,year,standard,genre,technique,source,date,price
0,Duet,김환기,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,김환기,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,김환기,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,김환기,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,김환기,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"


In [176]:
df.isnull().sum()

title           0
author          0
year         1607
standard        3
genre           0
technique      89
source          0
date            0
price           0
dtype: int64

# ARTPRICE

## Author

김환기 : 0, 
이우환 : 1, 
박서보 : 2, 
정상화 : 3, 
김창열 : 4, 
박수근 : 5, 
천경자 : 6, 
이중섭 : 7, 
윤형근 : 8, 
이대원 : 9, 
유영국 : 10, 
장욱진 : 11, 
김종학 : 12, 
백남준 : 13, 
오치균 : 14, 
하종현 : 15, 
도상봉 : 16, 
이강소 : 17, 
이왈종 : 18, 
정선 : 19, 
김홍도 : 20, 
김태호 : 21, 
이상범 : 22, 
남관 : 23, 
이응노 : 24, 
전광영 : 25, 
김정희 : 26, 
서세옥 : 27, 
이성자 : 28, 
김기창 : 29

In [177]:
df['author'] = df['author'].map({'김환기' : 0, '이우환' : 1, '박서보' : 2, '정상화' : 3, '김창열' : 4, '박수근' : 5, '천경자' : 6, 
                                        '이중섭' : 7, '윤형근' : 8, '이대원' : 9, '유영국' : 10, '장욱진' : 11, '김종학' : 12, '백남준' : 13, 
                                        '오치균' : 14, '하종현' : 15, '도상봉' : 16, '이강소' : 17, '이왈종' : 18, '정선' : 19, '김홍도' : 20, 
                                        '김태호' : 21, '이상범' : 22, '남관' : 23, '이응노' : 24, '전광영' : 25, '김정희' : 26, '서세옥' : 27, 
                                        '이성자' : 28, '김기창' : 29})
df = df.astype({'author':'int'})

In [178]:
df.head()

,title,author,year,standard,genre,technique,source,date,price
0,Duet,0,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,0,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,0,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,0,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"


## Standard

가로*세로 곱한 값을 가져와서 '넓이'로 만든다.
결측치 있는 것은 drop?

In [179]:
df[df['standard'].isnull()]

,title,author,year,standard,genre,technique,source,date,price
2980,Burunt Umber & Ultramarine Blue,8,NaN,NaN,회화,NaN,서울옥션 블루,2019/11/21,"￦90,000,000"
6168,하경산수,22,NaN,NaN,회화,종이에 수묵담채,마이아트옥션,2021/03/04,"￦16,000,000"
7013,고추,24,1979,NaN,회화,종이에 수묵담채,마이아트옥션,2017/05/22,"￦9,000,000"


결측치 데이터 3개 있어서 drop함

In [180]:
df.dropna(subset=['standard'], inplace=True)
df[df['standard'].isnull()]

,title,author,year,standard,genre,technique,source,date,price


데이터는 대개 다음과 같은 형태로 구성된다.

1. {숫자}x{숫자}cm : 대부분
2. {숫자}x{숫자}, {숫자}x{숫자}....
3. 지름 {숫자}cm
4. 지름 {숫자}x{숫자}

In [181]:
df = df[~df['standard'].str.contains('☓')]
df = df[~df['standard'].str.contains(':')]
df = df[~df['standard'].str.contains('지름')]
df = df[~df['standard'].str.contains('a.')]
df = df[~df['standard'].str.contains(',')]
df = df[~df['standard'].str.contains('외')]
df = df[~df['standard'].str.contains("\(")]
df = df[~df['standard'].str.contains("/")]
df = df[df['standard'].str.contains("×")]
df.reset_index(drop=True, inplace=True)
df

,title,author,year,standard,genre,technique,source,date,price
0,Duet,0,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,0,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,0,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,0,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"
...,...,...,...,...,...,...,...,...,...
8177,독수리,29,1940년대,116×71cm,회화,종이에 수묵담채,아트데이옥션 온라인,2016/11/29,"￦7,000,000"
8178,군상,29,1940년대,14.8×21.8cm,회화,"종이에 먹, 연필",아트데이옥션 온라인,2016/11/29,"￦340,000"
8179,부엉이,29,1940년대,65×62.5cm,판화,석판화,K옥션 온라인,2016/11/29,"￦300,000"
8180,삽화,29,1940년대,13×17.5cm,회화,종이에 수묵,에이옥션 온라인,2016/11/23,"￦140,000"


In [182]:
df['area']=0
for j in range(len(df)):
    text = df['standard'][j]
    height = text.split('×')[0]
    width = text.split('×')[1]
    height = float(height)
    if 'cm' in width:
        width = float(width[:-2])
    else:
        width = float(width)
    df['area'][j] = height*width

<ipython-input-182-849df69f0419>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area'][j] = height*width


## genre

In [183]:
df = df.replace(['회화', '판화', '서예', '서예-현판', '기타', '입체', '공예', '사진/미디어', '공예-도자기'],[0,1,2,3,4,5,6,7,8])
df.head()

,title,author,year,standard,genre,technique,source,date,price,area
0,Duet,0,NaN,79.5×56.7cm,1,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000",4507
1,27-VIII-70 #186,0,2015,100×72.7cm,1,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000",7270
2,무제,0,1969,120.5×85.5cm,0,코튼에 유채,K옥션,2021/06/23,"￦680,000,000",10302
3,27-XI-71 #211,0,1971,176.3×126.3cm,0,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000",22266
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,0,신문에 유채,서울옥션,2021/06/22,"￦72,000,000",2169


## technique

바탕재 : 목탄, 신문지, 종이, 직물, 캔버스, 하드보드, 한지, 도자기, 세라믹, 테라코타, 메소나이트, 나무, 낙엽, 은지

재료 : 과슈, 마카, 매직, 사인펜, 색연필, 수묵, 수묵담채, 수채, 연필, 유화, 잉크, 채색, 펜, 혼합재료, 안료, 그라파이트, 아크릴릭, 콩테, 흑연, 콜라주, 먹, 수묵채색, 새김, 크레파스, 에나멜, 오일파스텔

프린트, 스크린, 부채, 포스터, 탁본, 인쇄, 지본묵서, 판화, 접선, 장지, 신문 = 종이
카드보드 = 하드보드
드라이포인트,아쿼틴트, 에칭,믹소그라피아, 믹소그라피, 믹소그래프  = 금속
디아섹 = 아크릴
천, 비단, 세리그라프, 에코백, 면, 견, 마포, 마대, 모시, 린넨, 금견, 코튼, = 직물
도자, 세라믹 = 도자기
목판인쇄본 = 나무
켄버스 = 캔버스

유채 = 유화+채색


In [184]:
tech_dictionary = {'프린트': '종이', '스크린':'종이', '부채': '종이', '탁본' : '종이', '인쇄':'종이', '지본묵서': '종이',
             '판화': '종이', '접선':'종이', '지본수묵':'종이', '포스터':'종이','장지': '종이', '신문': '종이', '지도':'종이', 
            
            '카드보드':'하드보드', 
                   
             '드라이포인트': '금속', '아쿼틴트': '금속', '에칭': '금속','믹소그라피아': '금속','믹소그라피': '금속',
                   
            '믹소그래프': '금속','디아섹': '아크릴', 
            '천': '직물', '비단': '직물', '세리그라프': '직물','세리그래프':'직물','에코백': '직물', 
            '면': '직물', '견': '직물', '마포': '직물', '마대': '직물', '모시': '직물', '린넨': '직물', '금견': '직물', 
            '코튼': '직물', 
            '도자에': '세라믹', '도자기': '세라믹', '목판인쇄본': '나무', '켄버스':'캔버스',
            '그라파이트' : '흑연',
            '나뭇잎' : '낙엽',  
            '먹' : '수묵',
            '수묵채색' : '수묵담채',
            '유채':'유화 채색'}

tech_dictionary.items()

def tech_replace(text, dic):
    for i, j in dic.items():
        text = text.replace(i,j)
    return text

for i in range(len(df)):
    text = str(df['technique'][i])
#     print("Previous : ", text)
    new_text = tech_replace(text, tech_dictionary)
#     print(new_text)
    df['technique'][i] = new_text

<ipython-input-184-f9c270990f23>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['technique'][i] = new_text


In [185]:
base_material = ['목탄', '종이',  '직물',  '캔버스', '하드보드', '한지', 
                 '세라믹', '테라코타', '메소나이트', '나무', '낙엽', '은지', '금속']

material = ['과슈', '마카', '매직', '사인펜', '색연필', '수묵', '수묵담채', '수채', '연필', '유화', '잉크', 
            '채색', '펜', '혼합재료', '안료', '그라파이트', '아크릴', '콩테', '콜라주', '새김', 
            '크레파스', '에나멜', '오일파스텔']
arr = ['목탄', '종이', '직물', '캔버스', '하드보드', '한지','세라믹', '테라코타', '메소나이트', '나무', '낙엽', '은지', '금속',
       '과슈', '마카', '매직', '사인펜', '색연필', '수묵', '수묵담채', '수채', '연필', '유화', '유채', '잉크', '채색', '펜', 
       '혼합재료', '안료', '흑연', '아크릴', '콩테', '콜라주', '새김', '크레파스', '에나멜', '오일파스텔']
tech_iter = ['base_charcoal', 'base_paper', 'base_fiber', 'base_canvas', 'base_hardboard', 'base_hanji', 'base_ceramic',
            'base_terracotta', 'base_masonite', 'base_wood', 'base_leaf', 'base_silverpaper', 'base_metal',
            'mate_gouache', 'mate_maca', 'mate_magic', 'mate_signpen', 'mate_colorpencil', 'mate_korean_ink', 'mate_korean_ink_and_color',
            'mate_watercolor', 'mate_pencil', 'mate_oil', 'mate_oil_and_color', 'mate_ink', 'mate_color', 'mate_pen', 'mate_mixture',
            'mate_pigment', 'mate_graphite', 'mate_acrylic', 'mate_conte', 'mate_collage', 'mate_engrave', 'mate_crayon', 'mate_enamel',
            'mate_oilpastel']
print(len(arr))
print(len(tech_iter))

37
37


In [186]:
df['base_charcoal'] = 0
df['base_paper'] = 0
df['base_fiber'] = 0
df['base_canvas'] = 0
df['base_hardboard'] = 0
df['base_hanji'] = 0
df['base_ceramic'] = 0
df['base_terracotta'] = 0
df['base_masonite'] = 0
df['base_wood'] = 0
df['base_leaf'] = 0
df['base_silverpaper'] = 0
df['base_metal'] = 0

df['mate_gouache'] = 0
df['mate_maca'] = 0
df['mate_magic'] = 0
df['mate_signpen'] = 0
df['mate_colorpencil'] = 0
df['mate_korean_ink'] = 0
df['mate_korean_ink_and_color'] = 0
df['mate_watercolor'] = 0
df['mate_pencil'] = 0
df['mate_oil'] = 0
df['mate_oil_and_color'] = 0
df['mate_ink'] = 0
df['mate_color'] = 0
df['mate_pen'] = 0
df['mate_mixture'] = 0
df['mate_pigment'] = 0 #안료
df['mate_graphite'] = 0
df['mate_acrylic'] = 0
df['mate_conte'] = 0
df['mate_collage'] = 0
df['mate_engrave'] = 0 #새김
df['mate_crayon'] = 0 #크레파스
df['mate_enamel'] = 0
df['mate_oilpastel'] = 0

In [187]:
df_copy = df.copy()

In [188]:
for i in range(0, len(df_copy)):
    text = str(df_copy['technique'][i])
    for j in range(0, len(arr)):
        if arr[j] in text:
            df_copy[tech_iter[j]][i]=1
            continue

<ipython-input-188-f087a86ac1fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[tech_iter[j]][i]=1


In [189]:
df_copy['base_paper'].value_counts()

1    5342
0    2840
Name: base_paper, dtype: int64

In [190]:
for word in tech_iter:
    print(word)
    print(df_copy[word].value_counts())

base_charcoal
0    8160
1      22
Name: base_charcoal, dtype: int64
base_paper
1    5342
0    2840
Name: base_paper, dtype: int64
base_fiber
0    7428
1     754
Name: base_fiber, dtype: int64
base_canvas
0    6635
1    1547
Name: base_canvas, dtype: int64
base_hardboard
0    8155
1      27
Name: base_hardboard, dtype: int64
base_hanji
0    7863
1     319
Name: base_hanji, dtype: int64
base_ceramic
0    8168
1      14
Name: base_ceramic, dtype: int64
base_terracotta
0    8170
1      12
Name: base_terracotta, dtype: int64
base_masonite
0    8180
1       2
Name: base_masonite, dtype: int64
base_wood
0    8118
1      64
Name: base_wood, dtype: int64
base_leaf
0    8168
1      14
Name: base_leaf, dtype: int64
base_silverpaper
0    8176
1       6
Name: base_silverpaper, dtype: int64
base_metal
0    8089
1      93
Name: base_metal, dtype: int64
mate_gouache
0    8051
1     131
Name: mate_gouache, dtype: int64
mate_maca
0    8182
Name: mate_maca, dtype: int64
mate_magic
0    8171
1      11
Nam

In [191]:
df = df_copy

## source

K옥션 = 0 0
K옥션 온라인 = 0 1
서울옥션(홍콩, 블루) = 1 0
서울옥션 온라인 = 1 1
아트데이옥션 = 2 0
아트데이옥션 온라인 = 2 1
아이옥션 = 3 0
아이옥션 온라인 = 3 1
마이아트옥션 = 4 0
마이아트옥션 온라인 = 4 1
에이옥션 온라인 = 5 1
칸옥션 = 6 0
꼬모옥션 온라인 = 7 1

In [192]:
df['source'].value_counts()

K옥션 온라인       3101
K옥션           1378
서울옥션           870
아트데이옥션 온라인     701
에이옥션 온라인       509
서울옥션 온라인       449
아이옥션           253
서울옥션 홍콩        236
서울옥션 블루        145
아이옥션 온라인       132
마이아트옥션         126
아트데이옥션         102
칸옥션            101
꼬모옥션 온라인        68
마이아트옥션 온라인      11
Name: source, dtype: int64

In [193]:
df['online'] = 0
for i in range(0, len(df)):
    if '온라인' in df['source'][i]:
        df['online'][i] = 1

<ipython-input-193-daf944d43a37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['online'][i] = 1


In [194]:
df['online'].value_counts()

1    4971
0    3211
Name: online, dtype: int64

In [195]:
df = df.replace(['K옥션 온라인', 'K옥션'],[0,0])
df = df.replace(['서울옥션', '서울옥션 온라인', '서울옥션 홍콩', '서울옥션 블루'],[1, 1, 1, 1])
df = df.replace(['아트데이옥션 온라인', '아트데이옥션'],[2, 2])
df = df.replace(['아이옥션 온라인', '아이옥션'],[3, 3])
df = df.replace(['마이아트옥션 온라인', '마이아트옥션'],[4, 4])
df = df.replace(['에이옥션 온라인', '칸옥션', '꼬모옥션 온라인'],[5,6,7])

df['source'].value_counts()

0    4479
1    1700
2     803
5     509
3     385
4     137
6     101
7      68
Name: source, dtype: int64

## price

In [196]:
df['price']

0             ￦900,000
1             ￦850,000
2         ￦680,000,000
3       ￦3,050,000,000
4          ￦72,000,000
             ...      
8177        ￦7,000,000
8178          ￦340,000
8179          ￦300,000
8180          ￦140,000
8181        ￦1,800,000
Name: price, Length: 8182, dtype: object

In [199]:
for i in range(0, len(df)):
    text = df['price'][i]
    text = text.replace('￦','')
    text=  text.replace(',', '')
    df['price'][i] = text
df['price']

<ipython-input-199-01d8861cef3a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = text


0           900000
1           850000
2        680000000
3       3050000000
4         72000000
           ...    
8177       7000000
8178        340000
8179        300000
8180        140000
8181       1800000
Name: price, Length: 8182, dtype: object

## Ye